# Retail customer segmentation

The aim is to identify target customers for marketing strategies.

# Scope
* The data contains customer information from a mall's membership cards: Customer ID, age, gender, annual income, and spending score, a metric based on purchasing behavior. 

# Data exploration

## Dependencies

In [2]:
import sys
CONFIG_DIRECTORY = 'C:\\Users\\billy\\OneDrive\\Documents\\Python Scripts\\1. Portfolio\\mall-customer-segmentation\\mall-customer-segmentation'
if CONFIG_DIRECTORY not in sys.path:
    sys.path.insert(0, CONFIG_DIRECTORY)

import config
import datetime
import numpy as np
import pandas as pd
from IPython.display import display
from matplotlib import pyplot as plt
plt.style.use('seaborn-v0_8-muted')

## Import data

In [3]:
# Source: https://www.kaggle.com/datasets/vjchoudhary7/customer-segmentation-tutorial-in-python
raw_df = pd.read_csv(config.file_directory('raw') + 'Mall_Customers.csv')
print(f"Number of records: {raw_df.shape[0]}")
raw_df.head(3)

Number of records: 200


,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6


## Data types

In [5]:
raw_df.dtypes

CustomerID                 int64
Gender                    object
Age                        int64
Annual Income (k$)         int64
Spending Score (1-100)     int64
dtype: object

## Missing values

In [6]:
raw_df.isnull().sum()

CustomerID                0
Gender                    0
Age                       0
Annual Income (k$)        0
Spending Score (1-100)    0
dtype: int64

## Summary stats & distributions

In [ ]:
from custom_funcs import perc_func

# Data cleansing

# Feature engineering

# Model selection & training

# Model evaluation